## Insert a midi into here to sample it with whatever wav you load - given the wav's frequency


In [75]:
import librosa
from IPython.display import Audio
import matplotlib.mlab as mlab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
from typing import Tuple
import music21

In [76]:
#insert path to wav you want to sample with
wah_og, sampling_rate = librosa.load(r"C:\Users\g_bab\Downloads\waluigi_wahring2mob.wav", sr=44100, mono=True)
rest = np.zeros(len(wah_og))
len_scaler = len(wah_og)/12000
print(len_scaler)
Audio(wah_og, rate=sampling_rate)

3.0759166666666666


In [77]:
# def get_notes_chords_rests(instrument_type, path):
#     try:
#         midi = converter.parse(path)
#         parts = instrument.partitionByInstrument(midi)
#         note_list = []
#         for music_instrument in range(len(parts)):
#             if parts.parts[music_instrument].id in instrument_type:
#                 for element_by_offset in stream.iterator.OffsetIterator(parts[music_instrument]):
#                     for entry in element_by_offset:
#                         if isinstance(entry, note.Note):
#                             note_list.append(str(entry.pitch))
#                         elif isinstance(entry, chord.Chord):
#                             note_list.append('.'.join(str(n) for n in entry.normalOrder))
#                         elif isinstance(entry, note.Rest):
#                             note_list.append('Rest')
#         return note_list
#     except Exception as e:
#         print("failed on ", path)
#         pass

In [78]:
# notes = get_notes_chords_rests(keyboard_instruments, "Pirates_of_The_Caribbean.mid")

In [82]:
from music21 import converter, corpus, instrument, midi, note, chord, pitch

def open_midi(midi_path, remove_drums):
    # There is an one-line method to read MIDIs
    # but to remove the drums we need to manipulate some
    # low level MIDI events.
    mf = midi.MidiFile()
    mf.open(midi_path)
    mf.read()
    mf.close()
    if (remove_drums):
        for i in range(len(mf.tracks)):
            mf.tracks[i].events = [ev for ev in mf.tracks[i].events if ev.channel != 10]          

    return midi.translate.midiFileToStream(mf)

bun_midi = open_midi(r"C:\Users\g_bab\Downloads\Goodbye_Sengen.mid", True)  #replace with your own path
# bun_midi.show()
# bun_midi.show('midi')
print(type(bun_midi))

<class 'music21.stream.Score'>


In [83]:
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

def extract_notes(midi_part):
    parent_element = []
    ret = []
    count = 0
    for nt in midi_part.flat.notesAndRests:
        if count > 200:
            break
        if isinstance(nt, note.Note):
            ret.append(max(0.0, nt.pitch.ps))
            parent_element.append(nt)
        elif isinstance(nt, note.Rest):
            ret.append(0)
            parent_element.append(nt)
        elif isinstance(nt, chord.Chord):
            continue
        count+=1 
    return ret, parent_element
        
bun_notes = extract_notes(bun_midi)
print(bun_notes)
#print(bun_notes)
#print_parts_countour(bun_midi)

([50.0, 53.0, 55.0, 56.0, 57.0, 50.0, 48.0, 50.0, 0, 57.0, 50.0, 48.0, 50.0, 0, 53.0, 56.0, 53.0, 55.0, 53.0, 57.0, 50.0, 48.0, 50.0, 0, 57.0, 62.0, 60.0, 57.0, 55.0, 56.0, 57.0, 56.0, 57.0, 55.0, 53.0, 57.0, 50.0, 48.0, 50.0, 0, 57.0, 50.0, 48.0, 50.0, 0, 53.0, 56.0, 53.0, 55.0, 53.0, 57.0, 50.0, 48.0, 50.0, 0, 57.0, 62.0, 60.0, 57.0, 55.0, 53.0, 53.0, 0, 53.0, 55.0, 56.0, 57.0, 0, 55.0, 0, 57.0, 0, 55.0, 0, 57.0, 60.0, 57.0, 55.0, 57.0, 0, 55.0, 0, 53.0, 57.0, 0, 36.0, 48.0, 50.0, 48.0, 44.0, 43.0, 41.0, 50.0, 50.0, 50.0, 57.0, 50.0, 50.0, 50.0, 57.0, 50.0, 0, 50.0, 52.0, 53.0, 55.0, 57.0, 55.0, 53.0, 50.0, 48.0, 50.0, 53.0, 55.0, 50.0, 0, 57.0, 60.0, 62.0, 0, 57.0, 55.0, 53.0, 55.0, 57.0, 55.0, 53.0, 57.0, 65.0, 57.0, 0, 57.0, 55.0, 53.0, 55.0, 57.0, 55.0, 53.0, 50.0, 50.0, 50.0, 57.0, 50.0, 50.0, 50.0, 57.0, 50.0, 0, 48.0, 53.0, 52.0, 53.0, 56.0, 55.0, 53.0, 53.0, 53.0, 50.0, 50.0, 57.0, 0, 60.0, 57.0, 55.0, 57.0, 0, 59.0, 61.0, 63.0, 63.0, 63.0, 63.0, 64.0, 62.0, 61.0, 59.0, 59.0,

In [84]:
length = len(bun_notes[0])
wah_music = []
wahirginal = 261.63
for i in range(0, length):
    #print("bruh")
    note = bun_notes[1][i]
    #print(note.duration)
    if hasattr(note, 'pitch'):
        target_freq = note.pitch.frequency
        steps = 12*np.log(target_freq/wahirginal)/np.log(2)
        wah_note = librosa.effects.pitch_shift(wah_og, sampling_rate, n_steps=steps)
        if note.duration.type == 'complex':
            comp_list = note.duration.components
            for i in comp_list:
                wah_note_time = librosa.effects.time_stretch(wah_note, 1/(i.quarterLength)*len_scaler)
                wah_music.append(wah_note_time)
        else:
            wah_note_time = librosa.effects.time_stretch(wah_note,  1/(note.duration.quarterLength)*len_scaler)
            wah_music.append(wah_note_time)
    else:
        if note.duration.type == 'complex':
            comp_list = note.duration.components
            for i in comp_list:
                wah_note_time = librosa.effects.time_stretch(rest, 1/(i.quarterLength)*len_scaler)
                wah_music.append(wah_note_time)
        else:
            wah_note_time = librosa.effects.time_stretch(rest, 1/(note.duration.quarterLength)*len_scaler)
            wah_music.append(wah_note_time)
        
wah_concat = np.concatenate(wah_music)
Audio(wah_concat, rate=sampling_rate)